# Deepfake detection using GAN

In [2]:
# Imports
import numpy as np
import random
import torch

Controlling seeds to control reproducability.

In [ ]:
if False:
    seed = 0
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)